In [1]:
import numpy as np 
import pandas as pd 
import os
from tqdm import tqdm, trange
from sklearn.metrics import roc_auc_score
import pickle
import gc

In [2]:
os.listdir()

['sample_submission.csv',
 'jigsaw-unintended-bias-in-toxicity-classification.zip',
 'test.csv',
 'train.csv',
 '.~lock.train.csv#',
 'bias_toxic_comment.ipynb',
 '.ipynb_checkpoints']

In [3]:
BERT_MODEL = 'bert-base-uncased'
CASED = 'uncased' in BERT_MODEL
INPUT = '../input/jigsaw-bert-preprocessed-input/'
TEXT_COL = 'comment_text'
MAXLEN = 220

The training data is too large for my memory. So, instead of reading the whole file, I read it in chunks.

In [8]:
r2 = pd.read_csv('train.csv', nrows=10000)

chunk_count = 0
for chunk in reader:
    chunk_count = chunk_count + 1
print(chunk_count)    
df1 = reader.get_chunk(10)

In [9]:
r2.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [61]:
print(range(10))

range(0, 10)


In [ ]:
#Empty DataFrame
#df = pd.DataFrame()
#for i in range(19):
 #   r = range(0,(i+1)*10**5)
  #  chunk = pd.read_csv('train.csv', skiprows = r, nrows = 10**5 )
   # df.append(chunk)

#df = pd.DataFrame(reader.get_chunk(181))